In [10]:
%%writefile app.py
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import numpy as np
import base64
from PIL import Image
from io import BytesIO

app = Flask(__name__)

# Load the pre-trained Keras model
model = load_model('wafermap.keras')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    
    # Decode the base64 image data to an image
    images_data = data['images']  # Assume this is a list of base64 encoded strings
    images = np.array([decode_image(base64_str) for base64_str in images_data])

    # Extract numerical data from JSON
    numerical_data = np.array([data['dieSize'], data['waferMapDim_x'], data['waferMapDim_y']])
    
    # Reshape numerical data if necessary
    numerical_data = numerical_data.reshape(1, -1)  # Assuming a single prediction

    # Make prediction using the preprocessed image data and numerical data
    predictions = model.predict([images, numerical_data])
    return jsonify(predictions.tolist())

def decode_image(base64_str):
    # Decode the base64 string
    img_data = base64.b64decode(base64_str)
    # Convert binary data to image
    img = Image.open(BytesIO(img_data))
    # Convert image to grayscale
    img = img.convert('L')
    # Resize the image as required by your model
    img = img.resize((32, 32))
    img_array = np.array(img)
    # Ensure the image has the shape (32, 32, 1)
    img_array = img_array.reshape(32, 32, 1)
    return img_array

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

Overwriting app.py


In [12]:
%%writefile Dockerfile
# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 5000 available to the world outside this container
EXPOSE 5000

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]


Overwriting Dockerfile


In [14]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: flask-model-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      app: flask-model
  template:
    metadata:
      labels:
        app: flask-model
    spec:
      containers:
      - name: flask-model
        image: arvindprashanth:app
        ports:
        - containerPort: 5000


Overwriting deployment.yaml


In [16]:
%%writefile service.yaml
apiVersion: v1
kind: Service
metadata:
  name: flask-model-service
spec:
  type: LoadBalancer
  ports:
  - port: 5000
    targetPort: 5000
  selector:
    app: flask-model


Overwriting service.yaml
